## Imports

In [15]:
import pandas as pd 
import tabula 

## Get and parse data from WSEE Kraków

### Get data

In [16]:
df = tabula.read_pdf("https://wsse.krakow.pl/page/wp-content/uploads/2020/09/COVID-19-ogniska-stan-na-28.09.2020-MPWIS.pdf")
df = pd.DataFrame(df[0])

'pages' argument isn't specified.Will extract only from page 1 by default.


### Fix improper string parsing

In [17]:
def fix_str(item):
    if isinstance(item, str):
        return item.replace('\r', ' ').replace('1)', '').replace('3)', '')

def fix_series(series):
    return series.apply(fix_str)


df = df.apply(fix_series, axis=0)

### Change labels and trim table

In [18]:
df.iloc[1,0] = df.iloc[0,0]
df.iloc[1,1] = df.iloc[0,1]

df.columns = df.iloc[1]
df.drop(df.index[1], inplace=True)
df.drop(df.index[0], inplace=True)

In [19]:
df = df[['Powiat', 'Ogółem', 'Ludność', 'Liczba aktywnych przypadków']]
df

1,Powiat,Ogółem,Ludność,Liczba aktywnych przypadków
2,bocheński,477,106 858,37
3,brzeski,168,93 201,16
4,chrzanowski,126,124 536,37
5,dąbrowski,62,59 174,13
6,gorlicki,162,108 886,14
7,krakowski,608,279 239,117
8,limanowski,737,131 764,227
9,miechowski,97,48 818,35
10,myślenicki,576,127 600,196
11,nowosądecki,1 433,216 796,192
